In [1]:
from pymongo import MongoClient
from lxml import html

import os
import requests
import time
import re

mongo_connection_uri = os.environ.get("ME_CONFIG_MONGODB_URL")
client = MongoClient(mongo_connection_uri)

db = client["atcoder"]
contest_tasks_raw_collection = db["contest_tasks_raw"]
submissions_raw_collection = db["submissions_raw"]

In [2]:
def get_id(task_url):
    match = re.search(r'/contests/([^/]+)/tasks/([^/]+)', task_url)
    return match.group(1), match.group(2)

In [3]:
def get_html(contest_id, task_id):
    cookies = {
        '_ga': 'GA1.1.1669735030.1727884004',
        'REVEL_FLASH': '',
        'timeDelta': '1861',
        'REVEL_SESSION': '2dc8b5ca6727d90e3bbd1c5453e9c8d66db6d97e-%00w%3Afalse%00%00SessionKey%3Af712625f246d5b11787eeeac74b6cefd3f43ebfeb6e05e5a6b9568a08ba4367cebdfbc32acea8a-0b0fdeaa544c89f44b7326075ad509e6ba62c1c71fa969e5d0cab3c4e61ee30d%00%00UserScreenName%3Adarcy_zhang%00%00csrf_token%3AiecE%2FCD%2BWkZctrj%2BaFUUE0TovINExsqeelroJ6VJx9Q%3D%00%00_TS%3A1747914796%00%00UserName%3Adarcy_zhang%00%00a%3Afalse%00',
        '_ga_RC512FD18N': 'GS1.1.1732362767.18.1.1732362809.0.0.0',
    }

    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,en-GB;q=0.6,zh-TW;q=0.5',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'priority': 'u=0, i',
        'referer': f'https://atcoder.jp/contests/{contest_id}/submit',
        'sec-ch-ua': '"Microsoft Edge";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
    }

    url = f"https://atcoder.jp/contests/{contest_id}/submissions"

    params = {
        "f.Task": task_id,
        "f.LanguageName": "Python",
        "f.Status": "AC",
        "f.User": "",
    }

    response = requests.get(
        url,
        params=params,
        cookies=cookies,
        headers=headers,
    )
    response.encoding = "utf-8"
    return response.text

In [4]:
error_string = "500 Internal Server Error"
result = submissions_raw_collection.delete_many({"raw_html": {"$regex": error_string}})
print(f"Deleted {result.deleted_count} documents containing '{error_string}' in raw_html.")

for contest_tasks_raw_item in contest_tasks_raw_collection.find():
    tree = html.fromstring(contest_tasks_raw_item["raw_html"])
    tbody_list = tree.xpath("//tbody")
    if len(tbody_list) != 1:
        continue
    tr_list = tree.xpath("//tbody/tr")
    for tr_elem in tr_list:
        task_url = tr_elem.xpath(".//td[2]/a")[0].get("href")
        contest_id, task_id = get_id(task_url)
        existing_submissions = submissions_raw_collection.find_one(
            {"contest_id": contest_id, "task_id": task_id}
        )
        if existing_submissions is None:
            time.sleep(0.3)
            raw_html = get_html(contest_id, task_id)
            print(contest_id, task_id)
            submissions_raw_collection.insert_one(
                {"contest_id": contest_id, "task_id": task_id, "raw_html": raw_html}
            )
        else:
            # print(
            #     f"Submissions with {contest_id} {task_id} already exists, skipping insertion."
            # )
            pass

Deleted 14 documents containing '500 Internal Server Error' in raw_html.
abc356 abc356_d
abc343 abc343_e
abc330 abc330_e
abc329 abc329_d
abc324 abc324_d
abc323 abc323_b
abc321 abc321_c
abc315 abc315_d
abc314 abc314_c
abc313 abc313_d
abc305 abc305_b
abc302 abc302_c
abc302 abc302_d
abc294 abc294_c
abc291 abc291_e
abc285 abc285_c
abc284 abc284_b
abc284 abc284_f
abc281 abc281_e
abc279 abc279_d
abc272 abc272_b
abc268 abc268_c
abc266 abc266_c
abc265 abc265_c
abc264 abc264_c
abc261 abc261_c
abc233 abc233_e
abc232 abc232_a
abc216 abc216_e
abc215 abc215_c
abc214 abc214_d
abc214 abc214_e
abc212 abc212_e
abc211 abc211_c
abc208 abc208_d
abc207 abc207_d
abc203 abc203_d
abc202 abc202_d
abc201 abc201_d
zone2021 zone2021_a
zone2021 zone2021_c
abc199 abc199_d
jsc2021 jsc2021_a
abc198 abc198_d
abc198 abc198_e
abc197 abc197_d
abc195 abc195_d
abc194 abc194_d
abc193 abc193_d
arc113 arc113_a
abc192 abc192_e
abc191 abc191_c
abc191 abc191_e
abc188 abc188_d
abc188 abc188_f
abc187 abc187_b
abc186 abc186_c
abc18